### 来自廖雪峰官方网站
https://www.liaoxuefeng.com/wiki/897692888725344/923056337842176

Python的线程是真正的Posix Thread，而不是模拟出来的线程。

Python的标准库提供了两个模块：`thread`和`threading`，`thread`是低级模块，`threading`是高级模块，对`thread`进行了封装。绝大多数情况下，我们只需要使用`threading`这个高级模块。

启动一个线程就是把一个函数传入并创建`Thread`实例，然后调用`start()`开始执行：

In [2]:
import time
import threading

In [22]:
def loop():
    print("Thread {} is running.".format(threading.current_thread().name))
    n = 0
    while n < 5:
        n += 1
        print("Thread {} >>> {}".format(threading.current_thread().name, n))
        time.sleep(1)
    print("Thread {} ended".format(threading.current_thread().name))

t = threading.Thread(target=loop, name="LoopThread")  # 定义子线程，名字为 'LoopThread'
t.start()
t.join()

print("Thread {} ended".format(threading.current_thread().name))  # 主线程

Thread LoopThread is running.
Thread LoopThread >>> 1
Thread LoopThread >>> 2
Thread LoopThread >>> 3
Thread LoopThread >>> 4
Thread LoopThread >>> 5
Thread LoopThread ended
Thread MainThread ended


### Lock

多线程和多进程最大的不同在于：  
多进程中，同一个变量，各自有一份拷贝存在于每个进程中，互不影响。  
多线程中，所有变量都由所有线程共享，所以，任何一个变量都可以被任何一个线程修改。  
因此，线程之间共享数据最大的危险在于多个线程同时改一个变量，把内容给改乱了。

In [31]:
balance = 0

def change_it(n):
    global balance
    balance = balance + n
    balance = balance - n

def run_thread(n):
    for i in range(100000):
        change_it(n)

t1 = threading.Thread(target=run_thread, args=(5, ))
t2 = threading.Thread(target=run_thread, args=(8, ))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

5


修改`balance`需要多条语句，而执行这几条语句时，线程可能中断，从而导致多个线程把同一个对象的内容改乱了。  
如果我们要确保`balance`计算正确，就要给`change_it()`上一把锁，当某个线程开始执行`change_it()`时，我们说，该线程因为获得了锁，因此其他线程不能同时执行`change_it()`，只能等待，直到锁被释放后，获得该锁以后才能改。由于锁只有一个，无论多少线程，同一时刻最多只有一个线程持有该锁，所以，不会造成修改的冲突。创建一个锁就是通过`threading.Lock()`来实现：

In [37]:
balance = 0
lock = threading.Lock()

def run_thread(n):
    for i in range(100000):
        # 当多个线程同时执行lock.acquire()时，只有一个线程能成功地获取锁，然后继续执行代码，其他线程就继续等待直到获得锁为止。
        lock.acquire()  
        try:
            change_it(n)
        finally:
            # 获得锁的线程用完后一定要释放锁，否则那些等待锁的线程将永远等待下去，成为死线程
            lock.release()

t1 = threading.Thread(target=run_thread, args=(5, ))
t2 = threading.Thread(target=run_thread, args=(8, ))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


Python的线程虽然是真正的线程，但解释器执行代码时，有一个`GIL`锁：`Global Interpreter Lock`，任何Python线程执行前，必须先获得GIL锁，然后，每执行100条字节码，解释器就自动释放GIL锁，让别的线程有机会执行。这个GIL全局锁实际上把所有线程的执行代码都给上了锁，所以，**多线程在Python中只能交替执行，即使100个线程跑在100核CPU上，也只能用到1个核。**  

`GIL`是Python解释器设计的历史遗留问题，通常我们用的解释器是官方实现的`CPython`，要真正利用多核，除非重写一个不带`GIL`的解释器。  

所以，在Python中，可以使用多线程，但不要指望能有效利用多核。如果一定要通过多线程利用多核，那只能通过C扩展来实现，不过这样就失去了Python简单易用的特点。

不过，也不用过于担心，**Python虽然不能利用多线程实现多核任务，但可以通过多进程实现多核任务。**多个Python进程有各自独立的`GIL`锁，互不影响。